# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import itertools
import time #Tempo de cada modelo
import matplotlib.pyplot as plt
import cv2

from imblearn.pipeline import Pipeline
from plotly import figure_factory as ff
from matplotlib.colors import ListedColormap

#Transformação em imagens
from skimage.segmentation import felzenszwalb, slic, join_segmentations, mark_boundaries
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops, shape_index
from skimage.measure import label
from skimage.util import img_as_ubyte
#from scipy.stats import entropy
from skimage.filters.rank import entropy

#Métricas
from tensorflow.keras.metrics import MeanIoU
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

#Modelos
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

#Acesso ao Drive
#from google.colab import drive
#drive.mount('/content/drive')

# Código

### Pacote Superpixel

In [2]:
from os import listdir
from os.path import isfile, join
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage.segmentation import join_segmentations
from skimage import io
import numpy as np
import scipy as sp
from scipy.spatial import distance
import matplotlib.pyplot as plt
from datetime import datetime
from skimage.measure import label, regionprops
import math

import sys

def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data,axis=0)
    try:
        if not cov:
            cov = np.cov(data.T)
        inv_covmat = sp.linalg.inv(cov)
        left_term = np.dot(x_minus_mu, inv_covmat)
        mahal = np.dot(left_term, x_minus_mu.T)
        if type (mahal) is np.float64:
            return mahal
        return mahal.diagonal()
    except:
        return 9999999999999999999

def get_orderedsegments(label_img, max_pixels=25):
    uniques = np.unique(label_img, return_counts=True)
    filtered = [[],[]]
    filtered[0] = np.array(uniques[0][uniques[1]<=max_pixels])
    inds = filtered[0].argsort()
    sortedSegments = filtered[0][inds]
    return sortedSegments

def get_neighbors(label_img):
    #print(label_img.shape)
    props = regionprops(label_img)
    vizinhos_original = []
    vizinhos_original.append([])
    len_props=len(props)
    for i in range(0,len_props):
        #if (i)%int(len_props/5) == 0:
            #print(i)
        vizinhos_original.append([])
        for j in props[i].coords:
            if j[0]-1>=0:
                if props[i].label!=label_img[j[0]-1,j[1]]:
                    temp=label_img[j[0]-1,j[1]]
                    vizinhos_original[props[i].label].append(temp)
            if j[1]-1>=0:
                if props[i].label!=label_img[j[0],j[1]-1]:
                    vizinhos_original[props[i].label].append(label_img[j[0],j[1]-1])
            if j[0]+1<label_img.shape[0]:
                if props[i].label!=label_img[j[0]+1,j[1]]:
                    vizinhos_original[props[i].label].append(label_img[j[0]+1,j[1]])
            if j[1]+1<label_img.shape[1]:
                if props[i].label!=label_img[j[0],j[1]+1]:
                    vizinhos_original[props[i].label].append(label_img[j[0],j[1]+1])
        vizinhos_original[props[i].label] = list(np.unique(vizinhos_original[props[i].label]))
    #print(len_props)
    return vizinhos_original

def join_segments(label_img, img, min_number_pixels=50, distance='mean', verbose=True):
    if verbose:
        print('gerando labels unicos')
    label_img = label(label_img, connectivity=1)
    if verbose:
        print('gerando vizinhos unicos')
    vizinhos = get_neighbors(label_img)
    if verbose:
        print('gerando segmentos de tamanho minimo',min_number_pixels)
    t1=datetime.now()
    counter=0
    k=np.min(label_img)
    label_img_max=np.max(label_img)
    to_print = int(label_img_max/1000)
    if to_print<1000:
        to_print=1000
    while (k <= label_img_max):
        t2=datetime.now()
        i=k

        label_img_i_len = len(label_img[label_img==i])
        if (label_img_i_len>=min_number_pixels) or (label_img_i_len==0):
            k+=1
            continue

        minDistance = 9999999999999999999999
        minDistanceSeg = -1

        for v in vizinhos[i]:
            if v==i:
                continue
            if minDistanceSeg == -1:
                minDistanceSeg = v
            m=99999999999999999999999
            len_v = len(label_img[label_img==v])
            if len_v==0:
                continue
            if len_v>label_img_i_len and len_v>2:
                if distance=='mean':
                    m = mahalanobis(np.mean(img[label_img==i], axis=0),img[label_img==v])
                else:
                    m = mahalanobis(np.median(img[label_img==i], axis=0),img[label_img==v])

            elif label_img_i_len>2:
                if distance=='mean':
                    m = mahalanobis(np.mean(img[label_img==v], axis=0),img[label_img==i])
                else:
                    m = mahalanobis(np.median(img[label_img==v], axis=0),img[label_img==i])
            if m<minDistance:
                minDistance = m
                minDistanceSeg = v

        finalsegment = -1
        removedsegment = -1
        if minDistanceSeg<i:
            finalsegment = minDistanceSeg
            removedsegment = i
        else:
            finalsegment = i
            removedsegment = minDistanceSeg

        label_img[label_img==removedsegment] = finalsegment

        finalsegment_len = len(label_img[label_img==finalsegment])

        if finalsegment<=k and finalsegment_len<min_number_pixels:
            k=finalsegment
        else:
            k+=1

        vizinhos[finalsegment] = [*vizinhos[finalsegment],*vizinhos[removedsegment]]
        vizinhos[finalsegment] = list(np.unique(vizinhos[finalsegment]))
        temp = vizinhos[removedsegment].copy()
        for j in temp:
            try:
                vizinhos[j].remove(removedsegment)
            except:
                print('ERRO removendo:',removedsegment)
            vizinhos[j].append(finalsegment)

            vizinhos[j] = list(np.unique(vizinhos[j]))

        counter+=1
        if counter%to_print==0 or k > label_img_max:
            #print(label_img_max,'|',k,'-',i,'-',finalsegment, '-', removedsegment,"|", np.max(label_img), '|', datetime.now()-t1, datetime.now()-t2, "-", counter, '-', (datetime.now()-t1)/counter," | ",'final segment len: ',finalsegment_len)#, '|', len(vizinhos[finalsegment-1]))
            if verbose:
                print(label_img_max,'|',k,'|',distance,'|', datetime.now()-t1, datetime.now()-t2, "-", counter, '-', (datetime.now()-t1)/counter)
        if k > label_img_max:
            label_img = label(label_img, connectivity=1)
            unique_labels = np.unique(label_img, return_counts=True)
            smaller_than_min = len(unique_labels[1][unique_labels[1]<min_number_pixels])
            if smaller_than_min>0:
                if verbose:
                    print("reprocessing: ",smaller_than_min,'|',unique_labels[1][unique_labels[1]<min_number_pixels],"|",unique_labels[0][unique_labels[1]<min_number_pixels])
                k=np.min(label_img)
                label_img_max=np.max(label_img)
                vizinhos = get_neighbors(label_img)
            else:
                break

    label_img = label(label_img, connectivity=1)
    return label_img

def segment_slic(img, save_image_name_path, filename):
#     segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/1400), compactness=5, sigma=1, start_label=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_slic01.npz'),segments)
#     segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/700), compactness=5, sigma=1, start_label=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_slic02.npz'),segments)
#     segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/500), compactness=5, sigma=1, start_label=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_slic03.npz'),segments)
#     segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/350), compactness=5, sigma=1, start_label=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_slic04.npz'),segments)
#     segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/230), compactness=5, sigma=1, start_label=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_slic05.npz'),segments)
    segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/170), compactness=5, sigma=1, start_label=1)
    np.savez(save_image_name_path.replace('.tif','_segments_slic06.npz'),segments)
    print('SLIC done', filename)

def segment_quick(img, save_image_name_path, filename):
#     segments = quickshift(img, kernel_size=2, max_dist=50, ratio=0.5)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick01.npz'),segments)
    segments = quickshift(img, kernel_size=3, max_dist=50, ratio=0.5)
    np.savez(save_image_name_path.replace('.tif','_segments_quick02.npz'),segments)
#     segments = quickshift(img, kernel_size=4, max_dist=50, ratio=0.5)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick03.npz'),segments)
#     segments = quickshift(img, kernel_size=5, max_dist=50, ratio=0.5)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick04.npz'),segments)
    print('Quick done', filename)

def segment_felz(img, save_image_name_path, filename):
    segments = felzenszwalb(img, scale=50, sigma=0.5, min_size=50)
    np.savez(save_image_name_path.replace('.tif','_segments_fz01.npz'),segments)
    segments = felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
    np.savez(save_image_name_path.replace('.tif','_segments_fz02.npz'),segments)
#     segments = felzenszwalb(img, scale=200, sigma=0.5, min_size=50)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz03.npz'),segments)
    segments = felzenszwalb(img, scale=400, sigma=0.5, min_size=50)
    np.savez(save_image_name_path.replace('.tif','_segments_fz04.npz'),segments)
    segments = felzenszwalb(img, scale=50, sigma=0.5, min_size=100)
    np.savez(save_image_name_path.replace('.tif','_segments_fz05.npz'),segments)
    segments = felzenszwalb(img, scale=100, sigma=0.5, min_size=100)
    np.savez(save_image_name_path.replace('.tif','_segments_fz06.npz'),segments)
#     segments = felzenszwalb(img, scale=50, sigma=0.7, min_size=50)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz07.npz'),segments)
#     segments = felzenszwalb(img, scale=100, sigma=0.7, min_size=50)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz08.npz'),segments)
#     segments = felzenszwalb(img, scale=200, sigma=0.7, min_size=50)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz09.npz'),segments)
#     segments = felzenszwalb(img, scale=400, sigma=0.7, min_size=50)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz10.npz'),segments)
#     segments = felzenszwalb(img, scale=50, sigma=0.7, min_size=100)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz11.npz'),segments)
#     segments = felzenszwalb(img, scale=100, sigma=0.7, min_size=100)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz12.npz'),segments)
    print('Felzenszwalb done', filename)

def segment_felz_slic(original_img, chunk_size,distance,min_number_pixels, dataset, filename, save_image_name_path):
#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/2000), compactness=5, sigma=1, start_label=1), felzenszwalb(img, scale=200, sigma=0.7, min_size=200))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz_slic01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

    final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
    for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
        for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
            img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
            segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1500), compactness=5, sigma=1, start_label=1), felzenszwalb(img, scale=100, sigma=0.7, min_size=150))
            segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
            final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
    segments = label(final_segments, connectivity=1)
    np.savez(save_image_name_path.replace('.tif','_segments_fz_slic02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1000), compactness=5, sigma=1, start_label=1), felzenszwalb(img, scale=100, sigma=0.7, min_size=150))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz_slic03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

    final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
    for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
        for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
            img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
            segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/500), compactness=5, sigma=1, start_label=1), felzenszwalb(img, scale=50, sigma=0.7, min_size=100))
            segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
            final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
    segments = label(final_segments, connectivity=1)
    np.savez(save_image_name_path.replace('.tif','_segments_fz_slic04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

    print('Felzenszwalb-SLIC done', filename)

def segment_quick_slic(original_img, chunk_size,distance,min_number_pixels, dataset, filename, save_image_name_path):
#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/2000), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick_slic01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1500), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick_slic02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1000), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick_slic03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/500), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_quick_slic04_'+distance+'_'+str(min_number_pixels)+'.npz'), segments)
    print('Quick-SLIC done', filename)


def segment_felz_quick(original_img, chunk_size,distance,min_number_pixels, dataset, filename, save_image_name_path):
#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=2, max_dist=50, ratio=0.5))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz_quick01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

    final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
    for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
        for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
            img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
            segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=3, max_dist=50, ratio=0.5))
            segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
            final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
    segments = label(final_segments, connectivity=1)
    np.savez(save_image_name_path.replace('.tif','_segments_fz_quick02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

#     final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
#     for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
#         for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
#             img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
#             segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=4, max_dist=50, ratio=0.5))
#             segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
#             final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
#     segments = label(final_segments, connectivity=1)
#     np.savez(save_image_name_path.replace('.tif','_segments_fz_quick03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)

    final_segments = np.zeros((original_img.shape[0],original_img.shape[1]),dtype=np.int32)
    for k in range(0,math.ceil(final_segments.shape[0]/chunk_size)):
        for l in range(0,math.ceil(final_segments.shape[1]/chunk_size)):
            img = original_img[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size]
            segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
            segments = join_segments(segments, img, dataset, filename, min_number_pixels=min_number_pixels, distance=distance)
            final_segments[k*chunk_size:(k+1)*chunk_size,l*chunk_size:(l+1)*chunk_size] = segments+1+np.max(final_segments)
    segments = label(final_segments, connectivity=1)
    np.savez(save_image_name_path.replace('.tif','_segments_fz_quick04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
    print('Felzenszwalb-Quick done', filename)



### Código

In [3]:
def plot_and_save(plots, grid, sub_folder="", save_grid=False):
  """
  Save plots in the /home/images folder as pdf files
  plots: a dictionary with coords as keys (x,y)
          and with items of the form:
          {"img": your_image, "title": title_of_the_image, "opts", {dictionary}}
        Obs.: the "opts" is an optional key that passes extra paramenters to
          the "imshow" method

  grid: the grid layout you want your plot to have:
        Ex.: (2,2) -> 2 horizontal, 2 vertical,
             (4,1) -> 4 horizontal, 1 vertical

  sub_folder: if provided, a folder that will store your images
              inside of the home/images/ folder
              /home/images/folder

  save_grid: if True also saves the final grid plot as grid.pdf
  """
  import os
  os.makedirs(f"/home/images/{sub_folder}/", exist_ok=True)

  # helper function to handle custom ploting options
  def make_plot(obj, plot_value):
    if plot_value.get("opts"):
      obj.imshow(plot_value["img"], **plot_value["opts"])
    else:
      obj.imshow(plot_value["img"])
    obj.set_title(plot_value["title"])

  # saves each plot individually
  for axis in plots:
    fig2, ax2 = plt.subplots(figsize=(10, 10))
    make_plot(ax2, plots[axis])
    fig2.savefig(f"/home/images/{sub_folder}/axis{axis}.pdf", bbox_inches="tight")
    plt.close()

  # arrange the plots in the correct grid for plotting
  fig, ax = plt.subplots(*grid, figsize=(10, 10), sharex=True, sharey=True)
  for axis in plots:
    if grid == (1,1):
      make_plot(ax, plots[axis])
    elif 1 in grid:
      make_plot(ax[sum(axis)], plots[axis])
    else:
      make_plot(ax[axis], plots[axis])

    plt.tight_layout()

  # optionally saves the grid image
  if save_grid:
    fig.savefig(f"/home/images/{sub_folder}/grid.pdf", bbox_inches="tight")

  # plots the image
  plt.show()

#----------------------------EXEMPLO---------------------------
#axes = {
#    (0,0): {"img": mark_boundaries(img, superpixel_fz),"title":"Felzenszwalbs's method"},
#    (0,1): {"img": mark_boundaries(img, superpixel_slic), "title":'SLIC'},
#    (1,0): {"img": mark_boundaries(img, superpixel_final), "title": 'Final'},
#    (1,1): {"img": mask_img, "title": 'Máscara'}
#}
#
#plot_and_save(plots=axes, grid=(2,2), sub_folder="teste", save_grid=True)
#--------------------------------------------------------------

#Função para calcular as estatísticas do superpixel
def stats_img(image,label):
  superpixel_stats = []

  im_rp  = image.reshape((image.shape[0]*image.shape[1],image.shape[2]))
  sli_1d = np.reshape(label,-1)
  uni    = np.unique(sli_1d)
  uu_1 = uu_2 = uu_3 = uu_4 = uu_5 = np.zeros(im_rp.shape)

  for i in uni:
    loc        = np.where(sli_1d==i)[0]
    mean_sp    = np.mean(im_rp[loc,:],axis=0)
    uu_1[loc,:]  = mean_sp

  superpixel_mean = np.reshape(uu_1,[image.shape[0],image.shape[1],image.shape[2]]).astype('uint8')
  plt.imshow(superpixel_mean)
  plt.show()

  for i in uni:
    loc        = np.where(sli_1d==i)[0]
    std_sp     = np.std(im_rp[loc,:],axis=0)
    uu_2[loc,:]  = std_sp

  superpixel_std = np.reshape(uu_2,[image.shape[0],image.shape[1],image.shape[2]]).astype('uint8')

  for i in uni:
    loc        = np.where(sli_1d==i)[0]
    median_sp  = np.median(im_rp[loc,:],axis=0)
    uu_3[loc,:]  = median_sp

  superpixel_median = np.reshape(uu_3,[image.shape[0],image.shape[1],image.shape[2]]).astype('uint8')

  for i in uni:
    loc        = np.where(sli_1d==i)[0]
    max_sp     = np.max(im_rp[loc,:],axis=0)
    uu_4[loc,:]  = max_sp

  superpixel_max = np.reshape(uu_4,[image.shape[0],image.shape[1],image.shape[2]]).astype('uint8')

  for i in uni:
    loc        = np.where(sli_1d==i)[0]
    min_sp     = np.min(im_rp[loc,:],axis=0)
    uu_5[loc,:]  = min_sp

  superpixel_min = np.reshape(uu_5,[image.shape[0],image.shape[1],image.shape[2]]).astype('uint8')

  superpixel_stats.append(superpixel_mean)
  superpixel_stats.append(superpixel_std)
  superpixel_stats.append(superpixel_median)
  superpixel_stats.append(superpixel_max)
  superpixel_stats.append(superpixel_min)

  return superpixel_stats

def add_features(image,label_fz,label_SLIC):
  image_gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #image_gray = 255*image_gray
  indices     = shape_index(image_gray)
  stats_fz    = stats_img(image,label_fz)
  stats_slic  = stats_img(image,label_SLIC)
#  lbp_feature = local_binary_pattern(image_gray,5,2)
#  feature_vector = np.dstack((feature_vector,lbp_feature))
#  lbp_feature = local_binary_pattern(image_gray,20,15)
#  feature_vector = np.dstack((feature_vector,lbp_feature))
  lbp_feature = local_binary_pattern(image_gray,3,4)
  fd,hog_feature = hog(image,orientations=12,pixels_per_cell=(3,3),cells_per_block=(1,1),channel_axis=-1,visualize=True)
#  fd,hog_feature = hog(image,orientations=9,pixels_per_cell=(5,5),cells_per_block=(1,1),channel_axis=-1,visualize=True)
#  feature_vector = np.dstack((feature_vector,hog_feature))
#  fd,hog_feature = hog(image,orientations=9,pixels_per_cell=(7,7),cells_per_block=(1,1),channel_axis=-1,visualize=True)
#  feature_vector = np.dstack((feature_vector,hog_feature))
#  fd,hog_feature = hog(image,orientations=9,pixels_per_cell=(15,15),cells_per_block=(1,1),channel_axis=-1,visualize=True)
#  feature_vector = np.dstack((feature_vector,hog_feature))

  glcm = graycomatrix(image_gray,[4], [0],levels=256,symmetric=False,normed=True)
  glcm_feature = np.zeros((image.shape[0],image.shape[1],4))

  for j in range(0,image.shape[0]):
    for k in range(0,image.shape[1]):
      gray_level = image_gray[j,k]
      if k+1 < image_gray.shape[1]:
        gray_level2 = image_gray[j,k+1]
        glcm_feature[j,k,0] = glcm[gray_level,gray_level2,0,0]
      if k-1 >= 0:
        gray_level2 = image_gray[j,k-1]
        glcm_feature[j,k,1] = glcm[gray_level,gray_level2,0,0]
      if j+1 < image_gray.shape[0]:
        gray_level2 = image_gray[j+1,k]
        glcm_feature[j,k,2] = glcm[gray_level,gray_level2,0,0]
      if j-1 >= 0:
        gray_level2 = image_gray[j-1,k]
        glcm_feature[j,k,3] = glcm[gray_level,gray_level2,0,0]

  feature_vector = np.dstack((image,indices))

  feature_vector = np.dstack((feature_vector,lbp_feature))
  feature_vector = np.dstack((feature_vector,hog_feature))

  feature_vector = np.concatenate((feature_vector,glcm_feature), axis=2)

  feature_vector = np.concatenate((feature_vector,stats_fz[0]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_fz[1]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_fz[2]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_fz[3]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_fz[4]), axis=2)

  feature_vector = np.concatenate((feature_vector,stats_slic[0]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_slic[1]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_slic[2]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_slic[3]), axis=2)
  feature_vector = np.concatenate((feature_vector,stats_slic[4]), axis=2)

  plt.imshow(indices)
  plt.show()
  plt.imshow(hog_feature, cmap='gray')
  plt.show()
  plt.imshow(lbp_feature)
  plt.show()
  plt.imshow(glcm_feature[:, :, 0], cmap='gray')
  plt.show()

  return feature_vector

#Função para criar o Superpixel
def superpixel_image_AllBands(image, min_number_pixels = 17, verbose=False):
    work_image = np.zeros((image.shape[0],image.shape[1],3),dtype=float)
    work_image[:,:,0] = image[:,:,0]
    work_image[:,:,1] = image[:,:,1]
    work_image[:,:,2] = image[:,:,2]

    s2 = felzenszwalb(work_image, scale=1, sigma=0.8, min_size=int(min_number_pixels))
    if verbose:
        print(f'Felzenszwalb number of segments: {len(np.unique(s2))}')

    s1 = slic(work_image, n_segments=int((work_image.shape[0]*work_image.shape[1]/(min_number_pixels*4))), compactness=3, sigma=1, start_label=1)
    if verbose:
        print(f'SLIC number of segments: {len(np.unique(s1))}')

    return s1,s2,work_image

def train_test(mat, X, y):
    modelo = mat['modelo']
    nome = mat['nome']
    grid = mat['grid']

    print(f'Começando {nome}.')
    ini = time.time()
    pipe = Pipeline([('modelo', modelo)])

    gscv = GridSearchCV(
        pipe,
        param_grid = grid,
        scoring =['f1','jaccard'],
        refit='jaccard',
        cv = 5,
        verbose = True,
        n_jobs = 10,
    )

    rscv = gscv.fit(X,y)
    fim = time.time()

    print(f'Término {nome} | Score: {round(rscv.best_score_,3)} | Time: {rscv.refit_time_}')
    return {'name':nome, 'object': rscv}

#exclui setores fora da amostra
def exclui_setores_fora(image,setores,features,verbose=True):
  features_final_lista = []
  masks_final_lista = []

  for i in range(0,len(imagens_lista)):
    features_final = features[i][setores[i] == 0]
    masks_final = gt_lista[i][setores[i] == 0]
    features_final_lista.append(features_final)
    masks_final_lista.append(masks_final)

    if verbose == True:
      print(f'Length Final Features: {len(features_final_lista[i])}',
            f'Length Mask Vector: {len(masks_final_lista[i])}',
            f'Positives in Mask: {masks_final_lista[i].sum()}')

  return features_final_lista, masks_final_lista

In [ ]:
mapa_cores = {
    0: (0, 0, 0, 0),        # vazio
    1: (1, 0, 0, 1)         # vermelho
}

lista_cores = [mapa_cores[x] for x in [0,1]]
cmap_ = ListedColormap(lista_cores)

imagens_lista  = []
imagens_treino = []
gt_lista       = []
setores        = []

path="D:/Big Data & ODS - IC/images/Zona Sul/"

for i in range(1,6):
    img = Image.open(path+"img_"+str(i)+".png")
    img = np.asarray(img)
    imagens_lista.append(img)

    img_treino = Image.open(path+"img_treino_"+str(i)+".png")
    img_treino  = np.asarray(img_treino )
    imagens_treino.append(img_treino )

    gt = Image.open(path+"mask_img_"+str(i)+".png")
    gt = np.asarray(gt).copy()
    gt = gt[:,:,0]
    gt[gt<100]=0
    gt[gt>100]=1
    gt_lista.append(gt)

    set = Image.open(path+"setoresAGNS_fora_img_"+str(i)+".png")
    set = np.asarray(set).copy()
    set = set[:,:,0]
    set[set>0]=1

    print("setores",set.shape)
    print(np.unique(set))
    setores.append(set)

    plt.imshow(imagens_lista[-1])
    plt.show()
    plt.imshow(gt_lista[-1], cmap=cmap_)
    plt.show()
    plt.imshow(setores[-1],cmap=cmap_)
    plt.show()
    plt.imshow(imagens_treino[-1])
    plt.show()
    plt.imshow(imagens_treino[-1])
    plt.imshow(gt_lista[-1], cmap=cmap_, alpha=0.6)
    plt.show()

In [ ]:
superpixels_fz = []
superpixels_SLIC = []
features = []
for img in imagens_lista:
    superpixel_slic, superpixel_fz, img_segmentada = superpixel_image_AllBands(img,verbose=True)
    plt.imshow(mark_boundaries(img, superpixel_fz, mode='subpixel'))
    plt.show()
    plt.imshow(mark_boundaries(img, superpixel_slic, mode='subpixel'))
    plt.show()
    superpixels_fz.append(superpixel_fz)
    superpixels_SLIC.append(superpixel_slic)
    features.append(add_features(img,superpixel_fz,superpixel_slic))
for f in features:
    print(len(f), len(f[0]))

In [ ]:
features_final_lista,masks_final_lista  = exclui_setores_fora(imagens_lista,setores,features,verbose=True)

In [7]:
treino_X = []
treino_Y = []
teste_X = []
teste_Y = []

X_treino = list(itertools.chain(features_final_lista[0],features_final_lista[1],features_final_lista[2],features_final_lista[3]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(masks_final_lista[0],masks_final_lista[1],masks_final_lista[2],masks_final_lista[3]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = masks_final_lista[4]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features_final_lista[4]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#--------------------------
X_treino = list(itertools.chain(features_final_lista[1],features_final_lista[2],features_final_lista[3],features_final_lista[4]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(masks_final_lista[1],masks_final_lista[2],masks_final_lista[3],masks_final_lista[4]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = masks_final_lista[0]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features_final_lista[0]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#---------------------------
X_treino = list(itertools.chain(features_final_lista[2],features_final_lista[3],features_final_lista[4],features_final_lista[0]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(masks_final_lista[2],masks_final_lista[3],masks_final_lista[4],masks_final_lista[0]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = masks_final_lista[1]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features_final_lista[1]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#---------------------------
X_treino = list(itertools.chain(features_final_lista[3],features_final_lista[4],features_final_lista[0],features_final_lista[1]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(masks_final_lista[3],masks_final_lista[4],masks_final_lista[0],masks_final_lista[1]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = masks_final_lista[2]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features_final_lista[2]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#---------------------------
X_treino = list(itertools.chain(features_final_lista[4],features_final_lista[0],features_final_lista[1],features_final_lista[2]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(masks_final_lista[4],masks_final_lista[0],masks_final_lista[1],masks_final_lista[2]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = masks_final_lista[3]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features_final_lista[3]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)

In [8]:
modelos=[]

nome = 'RandomForest'
modelo = RandomForestClassifier(random_state=42) #44%
grid = {
    'modelo__max_depth':[int(x) for x in range(2,4)],
    'modelo__n_estimators': [int(x) for x in np.linspace(start=650, stop=1000, num=2)],
    'modelo__criterion': ('gini', 'entropy'),
    'modelo__class_weight': [{0:1, 1:2},{0:1, 1:4},{0:1, 1:6}],
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

nome = 'XGBoost'
modelo = XGBClassifier(objective='binary:logistic',device='cuda',subsample=0.5)
grid = {
    'modelo__max_depth': [int(x) for x in np.linspace(start=2, stop=5,num=3)],
    'modelo__n_estimators': [int(x) for x in np.linspace(start=650, stop=1000, num=4)],
    'modelo__learning_rate': [int(x) for x in np.linspace(start=0.2, stop=1,num=3)],
    'modelo__min_child_weight': [int(x) for x in np.linspace(start=2, stop=5,num=3)],
    'modelo__scale_pos_weight': [int(x) for x in np.linspace(start=2, stop=6,num=3)]
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

nome = 'LGBMClassifier'
modelo = lgb.LGBMClassifier(random_state=42,objective='binary')
grid = {
#    'modelo__num_leaves': [int(x) for x in np.linspace(start=2, stop=20,num=3)],
    'modelo__n_estimators': [int(x) for x in np.linspace(start=600, stop=1000, num=3)],
#    'modelo__reg_alpha': [0, 1e-1, 1],
#    'modelo__reg_lambda': [0, 1e-1, 1],
    'modelo__max_depth': [int(x) for x in np.linspace(start=2, stop=5,num=3)],
    'modelo__class_weight': [{0:1, 1:2},{0:1, 1:4},{0:1, 1:6}],
    'modelo__learning_rate': [int(x) for x in np.linspace(start=0.2, stop=1,num=3)],
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

nome = 'Kmeans'
modelo = KMeans(n_clusters=2,random_state=42)
grid = {
#    'modelo__n_clusters': [int(x) for x in np.linspace(start=1, stop=2, num=2)],
    'modelo__max_iter': [int(x) for x in np.linspace(start=50, stop=500, num=10)],
    'modelo__algorithm': ['lloyd', 'elkan'],
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

nome = 'GaussianNB'
modelo = GaussianNB()
grid = {
#    'modelo__n_features_in_': [int(x) for x in np.linspace(start=1, stop=20, num=3)],
#    'modelo__epsilon_': [int(x) for x in np.linspace(start=1, stop=50, num=5)],
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

nome = 'LogisticRegression'
modelo = LogisticRegression(random_state=42)
grid = {
#    'modelo__penalty': ['l2', 'l1', 'elasticnet'],
#    'modelo__solver': ['lbfgs', 'sag'],
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

nome = 'MLPClassifier'
modelo = MLPClassifier(random_state=42)
grid = {
    'modelo__max_iter': [int(x) for x in np.linspace(start=20, stop=100,num=3)],
}

modelos.append({'nome': nome, 'modelo': modelo, 'grid': grid})

In [ ]:
%%time
resultados = [train_test(mat, X_treino, Y_treino) for mat in modelos] #X_treino_pca

info_modelo = []
info_values = []
info_time = []

for i, v in enumerate(resultados):

    info_modelo.append(resultados[i]['name'])
    info_values.append(round(resultados[i]['object'].best_score_,3))
    info_time.append(resultados[i]['object'].refit_time_)

dados = pd.DataFrame({'Model': info_modelo, 'Score': info_values, 'Time': info_time})
fig = ff.create_table(dados, height_constant=20)
fig.show()

In [ ]:
resultado_folds = []
pred_folds = []

for j in range(0,len(treino_X)):
  info_modelo = []
#  info_time = []
  modelo_final = []
  pred_modelo = []

  resultado = pd.DataFrame(columns=['modelo','F1 Grid', 'Acurácia','F1 Score','Precisao', 'Recall'])
  resultado_roc = pd.DataFrame(columns=['classificadores', 'fpr','tpr','auc'])

  for i, v in enumerate(modelos):

      info_modelo.append(modelos[i]['nome'])
#      info_time.append(resultados[i]['object'].refit_time_)
#      info_values = round(resultados[i]['object'].best_score_,3)

#      model = resultados[i]['object'].best_estimator_
      model = modelos[i]['modelo']
      model.fit(treino_X[j], treino_Y[j]) #X_treino_pca

      y_pred = model.predict(teste_X[j]) #X_treino_pca

      acc = round(accuracy_score(teste_Y[j], y_pred),3)
      f1 = round(f1_score(teste_Y[j], y_pred),3)
      pc = round(precision_score(teste_Y[j], y_pred),3)
      re = round(recall_score(teste_Y[j], y_pred),3)

#      resultado = resultado._append({'modelo':resultados[i]['name'],
      resultado = resultado._append({'modelo':modelos[i]['nome'],
#                              'F1 Grid': info_values,
                              'Acurácia':acc,
                              'F1 Score':f1,
                              'Precisao':pc,
                              'Recall': re}, ignore_index=True)

#      modelo_final.append({'nome': resultados[i]['name'], 'modelo': model})
      modelo_final.append({'nome': modelos[i]['nome'], 'modelo': model})
      pred_modelo.append({'nome': modelo_final[i]['nome'], 'pred': y_pred})

      fpr, tpr, _ = roc_curve(teste_Y[j],  y_pred)
      auc = roc_auc_score(teste_Y[j], y_pred)
#      resultado_roc = resultado_roc._append({'classificadores':resultados[i]['name'],
      resultado_roc = resultado_roc._append({'classificadores':modelos[i]['nome'],
                                          'fpr':fpr,
                                          'tpr':tpr,
                                          'auc':auc}, ignore_index=True)

  resultado_folds.append(resultado)
  pred_folds.append(pred_modelo)
  resultado_roc.set_index('classificadores', inplace=True)
  fig = ff.create_table(resultado, height_constant=20)
  fig.show()

  fig = plt.figure(figsize=(12,6))
  for k in resultado_roc.index:
      plt.plot(resultado_roc.loc[k]['fpr'],
              resultado_roc.loc[k]['tpr'],
              label="{}, AUC={:.3f}".format(k, resultado_roc.loc[k]['auc']))

  plt.plot([0,1], [0,1], color='orange', linestyle='--')

  plt.xticks(np.arange(0.0, 1.1, step=0.1))
  plt.xlabel("Taxa de Falso Positivo", fontsize=15)

  plt.yticks(np.arange(0.0, 1.1, step=0.1))
  plt.ylabel("Taxa de Verdadeiro Positivo", fontsize=15)

  plt.title('Curva ROC', fontweight='bold', fontsize=15)
  plt.legend(prop={'size':13}, loc='lower right')

  #plt.savefig('roc_AGNS.png', format='png', dpi=100)
  plt.show()

In [ ]:
superpixels_fz = []
superpixels_SLIC = []
features = []
for img in imagens_treino:
    superpixel_slic, superpixel_fz, img_segmentada = superpixel_image_AllBands(img,verbose=True)
    plt.imshow(mark_boundaries(img, superpixel_fz, mode='subpixel'))
    plt.show()
    plt.imshow(mark_boundaries(img, superpixel_slic, mode='subpixel'))
    plt.show()
    superpixels_fz.append(superpixel_fz)
    superpixels_SLIC.append(superpixel_slic)
    features.append(add_features(img,superpixel_fz,superpixel_slic))
for f in features:
    print(len(f), len(f[0]))

In [34]:
treino_X = []
treino_Y = []
teste_X = []
teste_Y = []

X_treino = list(itertools.chain(features[1],features[2],features[3],features[4]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(gt_lista[1],gt_lista[2],gt_lista[3],gt_lista[4]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = gt_lista[0]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features[0]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#--------------------------
X_treino = list(itertools.chain(features[0],features[2],features[3],features[4]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(gt_lista[0],gt_lista[2],gt_lista[3],gt_lista[4]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = gt_lista[1]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features[1]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#---------------------------
X_treino = list(itertools.chain(features[1],features[3],features[4],features[0]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(gt_lista[1],gt_lista[3],gt_lista[4],gt_lista[0]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = gt_lista[2]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features[2]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#---------------------------
X_treino = list(itertools.chain(features[2],features[4],features[0],features[1]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(gt_lista[2],gt_lista[4],gt_lista[0],gt_lista[1]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = gt_lista[3]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features[3]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)
#---------------------------
X_treino = list(itertools.chain(features[3],features[0],features[1],features[2]))
X_treino = np.asarray(X_treino)
treino_X.append(X_treino)
Y_treino = list(itertools.chain(gt_lista[3],gt_lista[0],gt_lista[1],gt_lista[2]))
Y_treino = np.asarray(Y_treino)
treino_Y.append(Y_treino)

Y_teste = gt_lista[4]
Y_teste = np.asarray(Y_teste)
teste_Y.append(Y_teste)

X_teste = features[4]
X_teste = np.asarray(X_teste)
teste_X.append(X_teste)

In [35]:
for i in range(0,4):
    teste_X[i][np.isnan(teste_X[i])] = 0
    treino_X[i][np.isnan(treino_X[i])] = 0

In [ ]:
#Refazer a imagem
iou = MeanIoU(num_classes=2)
for i in range(0, len(imagens_treino)):
    for j in range(0,len(modelos)):
        modelo = modelos[j]['modelo']
        modelo.fit(treino_X[i].reshape((treino_X[i].shape[0]*treino_X[i].shape[1],treino_X[i].shape[2])),treino_Y[i].reshape((treino_Y[i].shape[0]*treino_Y[i].shape[1],1)).ravel())

        y_pred = modelo.predict(teste_X[i].reshape((teste_X[i].shape[0]*teste_X[i].shape[1],teste_X[i].shape[2])))
        
        fig, ax = plt.subplots(1,2, figsize=(10, 10), sharex=True, sharey=True)

        ax[0].imshow(gt_lista[i], cmap=cmap_)
        ax[0].set_title("Máscara")
        ax[1].imshow(np.reshape(y_pred,(img.shape[0],img.shape[1])), cmap=cmap_)
        ax[1].set_title(modelos[j]['nome'])

        plt.tight_layout()
        plt.show()

        iou.update_state(gt_lista[i].reshape((500*500)),y_pred)
        mIoU = iou.result().numpy()
        print("IoU",f"{modelos[j]['nome']} =", f"{mIoU*100:.2f}%")
#Fazer a média dos scores